In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import pickle

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Join tokens back together
    return ' '.join(tokens)

In [8]:
import pandas as pd

file_path = '/content/chatbot_dataset_full.csv'
df = pd.read_csv(file_path)

In [10]:
df.head()

,User Query,Chatbot Response,Intent
0,I want to return a product,You can return the product within 30 days of d...,Return_Request
1,My Kindle is not turning on,Please reset the device using the settings menu.,Technical_Support
2,I want to return a product,You can return the product within 30 days of d...,Return_Request
3,Can I update my delivery address?,You can change the address from 'Your Orders' ...,Change_Address
4,Can I initiate a return?,You can return the product within 30 days of d...,Return_Request


In [14]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
df['Processed_Query'] = df['User Query'].apply(preprocess_text)

In [16]:
# Encode the intents
le = LabelEncoder()
y = le.fit_transform(df['Intent'])

In [17]:
X = df['Processed_Query']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [19]:
# Train SVM model
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train_tfidf, y_train)

SVC(kernel='linear', probability=True)

In [20]:
# Make predictions
y_pred = svm_classifier.predict(X_test_tfidf)

In [21]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 1.0

Classification Report:
                      precision    recall  f1-score   support

        Account_Help       1.00      1.00      1.00      2095
        Cancel_Order       1.00      1.00      1.00      2031
      Change_Address       1.00      1.00      1.00      2196
    Exchange_Request       1.00      1.00      1.00      2273
      Gift_Card_Help       1.00      1.00      1.00      2157
        Order_Status       1.00      1.00      1.00      2079
       Payment_Issue       1.00      1.00      1.00      2323
    Prime_Membership       1.00      1.00      1.00      2340
     Product_Inquiry       1.00      1.00      1.00      2021
Product_Not_Received       1.00      1.00      1.00      2272
       Refund_Status       1.00      1.00      1.00      2212
      Return_Request       1.00      1.00      1.00      2121
    Shipping_Problem       1.00      1.00      1.00      2091
   Technical_Support       1.00      1.00      1.00      1789

            accuracy          

In [22]:
# Create a dictionary of intents to responses from the dataset
intent_response_dict = dict(zip(df['Intent'], df['Chatbot Response']))

In [23]:
def save_models():
    with open('tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf, f)
    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)
    with open('svm_model.pkl', 'wb') as f:
        pickle.dump(svm_classifier, f)
    with open('intent_response_dict.pkl', 'wb') as f:
        pickle.dump(intent_response_dict, f)

save_models()

In [24]:
def predict_intent(query):
    # Load saved models
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        loaded_tfidf = pickle.load(f)
    with open('label_encoder.pkl', 'rb') as f:
        loaded_le = pickle.load(f)
    with open('svm_model.pkl', 'rb') as f:
        loaded_svm = pickle.load(f)
    with open('intent_response_dict.pkl', 'rb') as f:
        loaded_responses = pickle.load(f)

    # Preprocess the query
    processed_query = preprocess_text(query)

    # Transform using TF-IDF
    query_tfidf = loaded_tfidf.transform([processed_query])

    # Predict intent
    intent_pred = loaded_svm.predict(query_tfidf)
    intent = loaded_le.inverse_transform(intent_pred)[0]

    # Get response
    response = loaded_responses.get(intent, "I'm not sure how to help with that. Please try rephrasing your question.")

    return response

# Test the chatbot
test_queries = [
    "I want to return a product",
    "My Kindle is not turning on",
    "Can I update my delivery address?",
    "Can I initiate a return?",
    "Where is my package?"
]

print("\nTesting Chatbot Responses:")
for query in test_queries:
    print(f"Query: {query}")
    print(f"Response: {predict_intent(query)}\n")

# Save the entire chatbot system as a class
class Chatbot:
    def __init__(self):
        with open('tfidf_vectorizer.pkl', 'rb') as f:
            self.tfidf = pickle.load(f)
        with open('label_encoder.pkl', 'rb') as f:
            self.le = pickle.load(f)
        with open('svm_model.pkl', 'rb') as f:
            self.svm = pickle.load(f)
        with open('intent_response_dict.pkl', 'rb') as f:
            self.responses = pickle.load(f)

    def respond(self, query):
        processed_query = preprocess_text(query)
        query_tfidf = self.tfidf.transform([processed_query])
        intent_pred = self.svm.predict(query_tfidf)
        intent = self.le.inverse_transform(intent_pred)[0]
        return self.responses.get(intent, "I'm not sure how to help with that. Please try rephrasing your question.")

# Save the chatbot instance
with open('chatbot.pkl', 'wb') as f:
    pickle.dump(Chatbot(), f)

# Example of loading and using the saved chatbot
with open('chatbot.pkl', 'rb') as f:
    loaded_chatbot = pickle.load(f)

print("Testing Loaded Chatbot:")
for query in test_queries:
    print(f"Query: {query}")
    print(f"Response: {loaded_chatbot.respond(query)}\n")


Testing Chatbot Responses:
Query: I want to return a product
Response: You can return the product within 30 days of delivery.

Query: My Kindle is not turning on
Response: Please reset the device using the settings menu.

Query: Can I update my delivery address?
Response: After shipping, the address can't be changed. Contact support for help.

Query: Can I initiate a return?
Response: You can return the product within 30 days of delivery.

Query: Where is my package?
Response: Please check with neighbors or contact support if not received.

Testing Loaded Chatbot:
Query: I want to return a product
Response: You can return the product within 30 days of delivery.

Query: My Kindle is not turning on
Response: Please reset the device using the settings menu.

Query: Can I update my delivery address?
Response: After shipping, the address can't be changed. Contact support for help.

Query: Can I initiate a return?
Response: You can return the product within 30 days of delivery.

Query: Wher

In [28]:
import pickle
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Chatbot class
class Chatbot:
    def __init__(self):
        with open('tfidf_vectorizer.pkl', 'rb') as f:
            self.tfidf = pickle.load(f)
        with open('label_encoder.pkl', 'rb') as f:
            self.le = pickle.load(f)
        with open('svm_model.pkl', 'rb') as f:
            self.svm = pickle.load(f)
        with open('intent_response_dict.pkl', 'rb') as f:
            self.responses = pickle.load(f)

    def respond(self, query):
        processed_query = preprocess_text(query)
        query_tfidf = self.tfidf.transform([processed_query])
        intent_pred = self.svm.predict(query_tfidf)
        intent = self.le.inverse_transform(intent_pred)[0]
        response = self.responses.get(intent, "I'm not sure how to help with that. Please try rephrasing your question.")
        return intent, response

# Chat interface with preliminary questions
def chat_interface():
    with open('chatbot.pkl', 'rb') as f:
        chatbot = pickle.load(f)

    print("=== Welcome to the Chatbot ===")
    print("Please answer a few questions to get started")
    print("--------------------------------------")

    # Collect user information
    user_info = {}

    # Question 1: Username
    user_info['username'] = input("What is your name? ")
    print(f"Chatbot: Nice to meet you, {user_info['username']}!\n")
    time.sleep(0.5)

    # Question 2: Product name
    user_info['product'] = input("What product are you inquiring about? ")
    print(f"Chatbot: Got it, you're asking about {user_info['product']}.\n")
    time.sleep(0.5)

    # Question 3: Order number
    user_info['order_number'] = input("What is your order number (if applicable)? ")
    if user_info['order_number']:
        print(f"Chatbot: Thanks for providing order number {user_info['order_number']}.\n")
    else:
        print("Chatbot: No order number provided, that's okay!\n")
    time.sleep(0.5)

    # Question 4: Issue type
    print("What type of issue are you facing?")
    print("1. Return | 2. Technical | 3. Delivery | 4. Other")
    issue_choice = input("Enter number (1-4): ")
    issue_types = {'1': 'Return', '2': 'Technical', '3': 'Delivery', '4': 'Other'}
    user_info['issue_type'] = issue_types.get(issue_choice, 'Other')
    print(f"Chatbot: Noted your issue type as {user_info['issue_type']}.\n")
    time.sleep(0.5)

    # Question 5: Preferred contact method
    user_info['contact_method'] = input("How would you prefer to be contacted if needed (email/phone)? ")
    print(f"Chatbot: We'll use {user_info['contact_method']} if we need to reach you.\n")
    time.sleep(0.5)

    print("--------------------------------------")
    print(f"Thanks {user_info['username']}! Now I can assist you with your {user_info['product']} queries.")
    print("Type 'quit' to exit the conversation")
    print("--------------------------------------")

    # Start the interactive chat with intent prediction
    while True:
        user_input = input(f"{user_info['username']}: ")
        if user_input.lower() == 'quit':
            print(f"Chatbot: Goodbye {user_info['username']}!")
            break

        intent, response = chatbot.respond(user_input)
        print(f"Chatbot [Intent: {intent}]: ", end="")
        time.sleep(0.5)
        for char in response:
            print(char, end='', flush=True)
            time.sleep(0.02)
        print("\n")

if __name__ == "__main__":
    chat_interface()

=== Welcome to the Chatbot ===
Please answer a few questions to get started
--------------------------------------
What is your name? zinna
Chatbot: Nice to meet you, zinna!

What product are you inquiring about? mobile
Chatbot: Got it, you're asking about mobile.

What is your order number (if applicable)? 765
Chatbot: Thanks for providing order number 765.

What type of issue are you facing?
1. Return | 2. Technical | 3. Delivery | 4. Other
Enter number (1-4): 2
Chatbot: Noted your issue type as Technical.

How would you prefer to be contacted if needed (email/phone)? 6369934324
Chatbot: We'll use 6369934324 if we need to reach you.

--------------------------------------
Thanks zinna! Now I can assist you with your mobile queries.
Type 'quit' to exit the conversation
--------------------------------------
zinna: How to reset my Fire TV Stick?
Chatbot [Intent: Technical_Support]: Please reset the device using the settings menu.

zinna: I want to return a product
Chatbot [Intent: Retu